In [1]:
import sys, os
sys.path.insert(0, "/home/gstupp/projects/wikidata-biothings/scheduled_bots")
sys.path.insert(0, "/home/gstupp/projects/WikidataIntegrator/")

In [2]:
import json
from collections import defaultdict
import pandas as pd
pd.options.display.max_colwidth = 0
from collections import OrderedDict
from scheduled_bots.ontology.DOID_obographs_bot import *
json_path='doid.json'

In [3]:
class Mesh:
    # https://meshb.nlm.nih.gov/record/ui?ui=D007822
    def __init__(self, mesh_path):
        self.d = json.load(open(mesh_path))
    def get_name(self, mesh_id):
        return self.d[mesh_id]['term']
    def get_parents(self, mesh_id):
        return {k for k,v in self.d.items() if any(t.rsplit(".",1)[0] in v.get('tree',[]) for t in self.d[mesh_id].get("tree", []))}
    def get_children(self, mesh_id):
        return {k for k,v in self.d.items() if any(t in map(lambda x: x.rsplit(".",1)[0], v.get('tree',[])) for t in self.d[mesh_id].get("tree", []))}
    def get_siblings(self, mesh_id):
        return {k for k,v in self.d.items() if any(t.rsplit(".",1)[0] in map(lambda x: x.rsplit(".",1)[0], v.get('tree',[])) for t in self.d[mesh_id].get("tree", []))} - {mesh_id}
    def format(self, mesh_id):
        # returns like: "D01234=Laryngeal Neoplasms"
        return "{}={}".format(mesh_id, self.d[mesh_id]['term'])
  

In [4]:
mesh_path = "/home/gstupp/projects/mesh/mesh.json"
m = Mesh(mesh_path)
#m.d['D018317']

In [5]:
with open(json_path) as f:
    d = json.load(f)
graphs = {g['id']: g for g in d['graphs']}
graph = graphs['http://purl.obolibrary.org/obo/doid.owl']
do = DOGraph(graph, None, True)

In [6]:
d = defaultdict(set)
for node in do.nodes.values():
    for xref in node.xrefs:
        d[xref].add(node.id)

In [7]:
dupes = sorted({k:v for k,v in d.items() if len(v)>1}.items(), key=lambda x:len(x[1]), reverse=True)
len(dupes)

1496

In [98]:
dupe_key = defaultdict(dict)
for k,v in dupes:
    prefix, _id = k.split(":", 1)
    dupe_key[prefix][_id] = v

In [100]:
{k:len(v) for k,v in dupe_key.items()}

{'CSP': 1,
 'ICD10CM': 280,
 'ICD9CM': 35,
 'ICDCM10': 1,
 'KEGG': 2,
 'MESH': 200,
 'NCI': 37,
 'OMIM': 46,
 'ORDO': 40,
 'SNOMEDCT_US_2016_03_01': 803,
 'UMLS_CUI': 51}

In [101]:
dupe_key['MESH']

{'C535952': {'http://purl.obolibrary.org/obo/DOID_4030',
  'http://purl.obolibrary.org/obo/DOID_4031'},
 'C537743': {'http://purl.obolibrary.org/obo/DOID_0050534',
  'http://purl.obolibrary.org/obo/DOID_8498'},
 'C537923': {'http://purl.obolibrary.org/obo/DOID_0060821',
  'http://purl.obolibrary.org/obo/DOID_14711'},
 'C562924': {'http://purl.obolibrary.org/obo/DOID_0060256',
  'http://purl.obolibrary.org/obo/DOID_0060258'},
 'C563020': {'http://purl.obolibrary.org/obo/DOID_6126',
  'http://purl.obolibrary.org/obo/DOID_7173'},
 'D000168': {'http://purl.obolibrary.org/obo/DOID_12960',
  'http://purl.obolibrary.org/obo/DOID_14705',
  'http://purl.obolibrary.org/obo/DOID_14768'},
 'D000230': {'http://purl.obolibrary.org/obo/DOID_299',
  'http://purl.obolibrary.org/obo/DOID_4903',
  'http://purl.obolibrary.org/obo/DOID_4929',
  'http://purl.obolibrary.org/obo/DOID_5675'},
 'D000236': {'http://purl.obolibrary.org/obo/DOID_3172',
  'http://purl.obolibrary.org/obo/DOID_5403',
  'http://purl.o

In [102]:
from IPython.display import display, HTML
def pretty_print(df):
    display( HTML( df.to_html().replace("\\n","<br>") ) )

In [111]:
def make_table(dk):
    ods = []
    for mesh_id, doids in dk.items():
        mesh_id = mesh_id.replace("MESH:", "")
        od = OrderedDict([('mesh_id', mesh_id),
                 ('mesh_term', m.d[mesh_id]['term'] if "term" in m.d[mesh_id] else m.d[mesh_id]['name']),
                 ('mesh_synonyms', "\n".join(m.d[mesh_id]['synonyms'])),
                 #('mesh_parents', ";".join(m.format(x) for x in m.get_parents(mesh_id))),
                 #('mesh_children', ";".join(m.format(x) for x in m.get_children(mesh_id))),
                 #('mesh_siblings', ";".join(m.format(x) for x in m.get_siblings(mesh_id))),
                 ("doid_lbls", "\n".join("{} ({})".format(do.nodes[x].lbl,x) if x in do.nodes else "*" for x in doids ))])
        ods.append(od)
    pretty_print(pd.DataFrame(ods))
make_table(dupe_key['MESH'])

,mesh_id,mesh_term,mesh_synonyms,doid_lbls
0,D020339,Optic Nerve Glioma,"Astrocytoma, Malignant, Optic NerveGlioma, Optic NerveMalignant Astrocytoma, Optic NerveAdult Optic Nerve GliomaChildhood Optic Nerve GliomaGlioma, Optic Nerve, AdultGlioma, Optic Nerve, ChildhoodMalignant Optic Nerve AstrocytomaOptic GliomaOptic Nerve Glioma, AdultOptic Nerve Glioma, ChildhoodOptic Nerve, Malignant AstrocytomaGlioma, OpticGliomas, OpticGliomas, Optic NerveOptic GliomasOptic Nerve Gliomas",childhood optic nerve glioma (http://purl.obolibrary.org/obo/DOID_6576)optic nerve glioma (http://purl.obolibrary.org/obo/DOID_4992)
1,D020936,"Epilepsy, Benign Neonatal","Neonatal Convulsions, BenignBenign Familial Infantile ConvulsionsBenign Familial Infantile Convulsions SyndromeBenign Familial Neonatal ConvulsionsBenign Familial Neonatal EpilepsyBenign Familial Neonatal-Infantile SeizuresBenign Infantile Familial ConvulsionsBenign Neonatal ConvulsionsBenign Neonatal EpilepsyBenign Neonatal Epilepsy, NonfamilialBenign Neonatal Non-Familial ConvulsionsBenign Neonatal Nonfamilial EpilepsyBenign Non-Familial Neonatal ConvulsionsConvulsions Benign Familial NeonatalConvulsions, Benign Familial Infantile, 1Convulsions, Benign Familial Infantile, 3Convulsions, Benign Neonatal, FamilialConvulsions, Benign Neonatal, Non-FamilialEpilepsy, Benign Neonatal, FamilialEpilepsy, Benign Neonatal, Non-FamilialEpilepsy, Benign Neonatal, NonfamilialEpilepsy, Benign Neonatal-InfantileFamilial Benign Neonatal ConvulsionsFamilial Benign Neonatal EpilepsyNon-Familial Benign Neonatal ConvulsionsNon-Familial Benign Neonatal EpilepsySeizures, Benign Familial Infantile, 1Seizures, Benign Familial Infantile, 3Seizures, Benign Familial Neonatal-InfantileBenign Familial Neonatal Infantile SeizuresBenign Neonatal ConvulsionBenign Neonatal EpilepsiesBenign Neonatal Non Familial ConvulsionsBenign Neonatal-Infantile EpilepsiesBenign Neonatal-Infantile EpilepsyBenign Non Familial Neonatal ConvulsionsConvulsion, Benign NeonatalConvulsions, Benign NeonatalEpilepsies, Benign NeonatalEpilepsies, Benign Neonatal-InfantileEpilepsy, Benign Neonatal InfantileNeonatal Convulsion, BenignNeonatal Epilepsies, BenignNeonatal Epilepsy, BenignNeonatal-Infantile Epilepsies, BenignNeonatal-Infantile Epilepsy, BenignNon Familial Benign Neonatal ConvulsionsNon Familial Benign Neonatal EpilepsySeizures, Benign Familial Neonatal Infantile",benign neonatal seizures (http://purl.obolibrary.org/obo/DOID_14264)benign familial neonatal epilepsy (http://purl.obolibrary.org/obo/DOID_14777)
2,D010854,Piedra,"Black PiedraWhite PiedraBlack PiedrasPiedra, BlackPiedra, WhitePiedrasPiedras, BlackPiedras, WhiteWhite Piedras",black piedra (http://purl.obolibrary.org/obo/DOID_12711)superficial mycosis (http://purl.obolibrary.org/obo/DOID_0050133)white piedra (http://purl.obolibrary.org/obo/DOID_13902)
3,D007174,"Disruptive, Impulse Control, and Conduct Disorders","Impulse Control DisordersIntermittent Explosive DisorderKleptomaniaImpulse-Control DisordersDisorders, Impulse ControlDisorders, Intermittent ExplosiveExplosive Disorder, IntermittentIntermittent Explosive Disorders",intermittent explosive disorder (http://purl.obolibrary.org/obo/DOID_12401)impulse control disorder (http://purl.obolibrary.org/obo/DOID_10937)kleptomania (http://purl.obolibrary.org/obo/DOID_12400)
4,D009494,Neurosyphilis,"General ParesisJuvenile ParesisParetic NeurosyphilisSyphilis, Central Nervous SystemCentral Nervous System SyphilisGeneral ParalysisGeneral Paresis of the InsaneNeurosyphilis, AsymptomaticNeurosyphilis, GummatousNeurosyphilis, JuvenileNeurosyphilis, SecondaryNeurosyphilis, SymptomaticParalysis, GeneralSecondary NeurosyphilisSyphilis, CNSAsymptomatic NeurosyphilisGeneral ParalysesGeneral ParesesGummatous NeurosyphilisJuvenile NeurosyphilisJuvenile ParesesNeurosyphilis, PareticParalyses, GeneralPareses, GeneralPareses, JuvenileParesis, GeneralParesis, Juvenile",late congenital syphilis (http://purl.obolibrary.org/obo/DOID_10039)asy

In [112]:
# try to reconcile mesh synonyms with DO names
def decomma(s):
    # Convexity Meningioma, Cerebral -> Cerebral Convexity Meningioma
    if "," in s:
        beg,end = map(str.strip, s.rsplit(",", 1))
        return end + " " + beg
    else:
        return s
def depluralize(s):
    s = s[:-1] if s.endswith("s") else s
    s = s[:-2] if s.endswith("'s") else s
    return s
def normalize_synonyms(syns: set):
    syns.update({decomma(s) for s in syns})
    syns.update({depluralize(s) for s in syns})
    syns = set(x.replace(" ", "").replace("'", "").replace(",", "").replace("-", "").lower() for x in syns)
    return syns
def normalize_synonym(s: str):
    return depluralize(decomma(s)).replace(" ", "").replace("'", "").replace(",", "").replace("-", "").lower()
normalize_synonyms({'a', 'bbbs', 'bs, a', "sadf's"})        

{'a', 'ab', 'abs', 'bbb', 'bbbs', 'bsa', 'sadf', 'sadfs'}

In [113]:
look_at = dict()
for mesh_id, doids in dupe_key['MESH'].items():
    mesh_id = mesh_id.replace("MESH:", "")
    mesh_term = m.d[mesh_id]['term'] if "term" in m.d[mesh_id] else m.d[mesh_id]['name']
    synonyms = normalize_synonyms({mesh_term} | set(m.d[mesh_id]['synonyms']))
    do_lbls = {k:normalize_synonym(do.nodes[k].lbl) for k in doids}
    lbls_do = {v:k for k,v in do_lbls.items()}
    if not all(x in synonyms for x in do_lbls.values()):
        missing = set(do_lbls.values())-synonyms
        look_at[mesh_id] = [lbls_do[m] for m in missing] + ["*"] + list(dupe_key['MESH'][mesh_id])

In [114]:
make_table(look_at)

,mesh_id,mesh_term,mesh_synonyms,doid_lbls
0,D010871,Pinealoma,"Neoplasms, PinealPineal NeoplasmsPinealocytomaPineoblastomaPineocytomaMixed Pineocytoma-PineoblastomaPineal Gland TumorPineal Parenchymal TumorsPineal TumorsMixed Pineocytoma PineoblastomaMixed Pineocytoma-PineoblastomasNeoplasm, PinealPineal Gland TumorsPineal NeoplasmPineal Parenchymal TumorPineal TumorPinealocytomasPinealomasPineoblastomasPineocytoma-Pineoblastoma, MixedPineocytoma-Pineoblastomas, MixedPineocytomasTumor, PinealTumor, Pineal GlandTumor, Pineal ParenchymalTumors, PinealTumors, Pineal GlandTumors, Pineal Parenchymal",pineal gland cancer (http://purl.obolibrary.org/obo/DOID_5032)*pineoblastoma (http://purl.obolibrary.org/obo/DOID_1664)pineal gland cancer (http://purl.obolibrary.org/obo/DOID_5032)
1,D008269,Macular Edema,"Central Retinal Edema, CystoidCystoid Macular EdemaMacular Edema, CystoidCystoid Macular DystrophyCystoid Macular Edema, PostoperativeIrvine-Gass SyndromeMacular Dystrophy, Dominant CystoidEdema, Cystoid MacularEdema, MacularIrvine Gass SyndromeSyndrome, Irvine-Gass",macular retinal edema (http://purl.obolibrary.org/obo/DOID_4449)*macular retinal edema (http://purl.obolibrary.org/obo/DOID_4449)cystoid macular edema (http://purl.obolibrary.org/obo/DOID_4447)
2,D013981,Tic Disorders,"Post-Traumatic Tic DisorderTic Disorder, Chronic Motor or VocalTic Disorder, Post-TraumaticTic Disorder, TransientTic Disorders, VocalVocal Tic DisordersChildhood Tic DisordersChronic Motor or Vocal Tic DisorderMotor Tic DisordersMotor or Vocal Tic Disorder, ChronicTic Disorders, ChildhoodTic Disorders, MotorTransient Tic DisorderChildhood Tic DisorderMotor Tic DisorderPost Traumatic Tic DisorderPost-Traumatic Tic DisordersTic DisorderTic Disorder, ChildhoodTic Disorder, MotorTic Disorder, Post TraumaticTic Disorder, VocalTic Disorders, Post-TraumaticTic Disorders, TransientTransient Tic DisordersVocal Tic Disorder",chronic tic disorder (http://purl.obolibrary.org/obo/DOID_10600)*transient tic disorder (http://purl.obolibrary.org/obo/DOID_2768)tic disorder (http://purl.obolibrary.org/obo/DOID_2769)chronic tic disorder (http://purl.obolibrary.org/obo/DOID_10600)
3,D015179,Colorectal Neoplasms,"Colorectal CancerColorectal CarcinomaColorectal TumorsNeoplasms, ColorectalCancer, ColorectalCancers, ColorectalCarcinoma, ColorectalCarcinomas, ColorectalColorectal CancersColorectal CarcinomasColorectal NeoplasmColorectal TumorNeoplasm, ColorectalTumor, ColorectalTumors, Colorectal",large intestine cancer (http://purl.obolibrary.org/obo/DOID_5672)*large intestine cancer (http://purl.obolibrary.org/obo/DOID_5672)colorectal cancer (http://purl.obolibrary.org/obo/DOID_9256)
4,D020936,"Epilepsy, Benign Neonatal","Neonatal Convulsions, BenignBenign Familial Infantile ConvulsionsBenign Familial Infantile Convulsions SyndromeBenign Familial Neonatal ConvulsionsBenign Familial Neonatal EpilepsyBenign Familial Neonatal-Infantile SeizuresBenign Infantile Familial ConvulsionsBenign Neonatal ConvulsionsBenign Neonatal EpilepsyBenign Neonatal Epilepsy, NonfamilialBenign Neonatal Non-Familial ConvulsionsBenign Neonatal Nonfamilial EpilepsyBenign Non-Familial Neonatal ConvulsionsConvulsions Benign Familial NeonatalConvulsions, Benign Familial Infantile, 1Convulsions, Benign Familial Infantile, 3Convulsions, Benign Neonatal, FamilialConvulsions, Benign Neonatal, Non-FamilialEpilepsy, Benign Neonatal, FamilialEpilepsy, Benign Neonatal, Non-FamilialEpilepsy, Benign Neonatal, NonfamilialEpilepsy, Benign Neonatal-InfantileFamilial Benign Neonatal ConvulsionsFamilial Benign Neonatal EpilepsyNon-Familial Benign Neonatal ConvulsionsNon-Familial Benign Neonatal EpilepsySeizures, Benign Familial Infantile, 1Seizures, Benign Familial Infantile, 3Seizures, Benign Familial Neonatal-InfantileBenign Familial Neonatal Infantile SeizuresBenign Neonatal ConvulsionBenign Neonatal EpilepsiesBenign Neonatal Non Familial ConvulsionsBenign Neonatal-Infantile EpilepsiesBenign Neonatal-Infantile Epileps

In [115]:
len(look_at)

74